In [ ]:
from pathlib import Path

import numpy as np
import struct
import matplotlib.pyplot as plt
import seaborn as sns

from topostats.plottingfuncs import Colormap
from topostats.filters import Filters
from topostats.io import read_yaml

colormap = Colormap()
cmap = colormap.get_cmap()


def open_asd(fname):
    with open(fname, "rb") as asd_file:
        # Read header information
        header = {}
        header["fileVersion"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]
        print(f"file version: {header['fileVersion']}")

        header["fileHeaderSize"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["frameHeaderSize"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["encNumber"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["operationNameSize"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["commentSize"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["dataTypeCh1"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["dataTypeCh2"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["numberFramesRecorded"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["numberFramesCurrent"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["scanDirection"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["fileName"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["xPixel"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["yPixel"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["xScanRange"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["yScanRange"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["avgFlag"] = np.fromfile(asd_file, dtype=bool, count=1)[0]

        header["avgNumber"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["yearRec"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["monthRec"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["dayRec"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["hourRec"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["minuteRec"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["secondRec"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["xRoundDeg"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["yRoundDeg"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["frameAcqTime"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["sensorSens"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["phaseSens"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["offset"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]  # Offset

        header["reg12byte"] = asd_file.read(12)

        header["machineNum"] = header["machineNum"] = struct.unpack("<i", asd_file.read(4))[0]

        header["adRange"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["adRes"] = np.fromfile(asd_file, dtype=np.int32, count=1)[0]

        header["xMaxScanRange"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["yMaxScanRange"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["xExtCoef"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["yExtCoef"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["zExtCoef"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["zDriveGain"] = np.fromfile(asd_file, dtype=np.float32, count=1)[0]

        header["operName"] = asd_file.read(header["operationNameSize"]).decode("utf-8")

        header["comment"] = asd_file.read(header["commentSize"]).decode("utf-8")

        header["AA"] = header["operName"]
        header["BB"] = header["comment"]

        frame_numbers = []
        frame_max_data = []
        frame_min_data = []
        x_offsets = []
        data_types = []
        x_tilts = []
        y_tilts = []
        flag_laser_ir = []

        # Create a 3D array to store the images
        preIm = np.zeros((header["numberFramesCurrent"], header["yPixel"], header["xPixel"]), dtype=np.int16)
        print(f"shape preIm on creation: {preIm.shape}")

        for k in range(header["numberFramesCurrent"]):
            # Frame header structure
            frame_numbers.append(struct.unpack("<i", asd_file.read(4))[0])  # Frame number
            frame_max_data.append(struct.unpack("<h", asd_file.read(2))[0])  # Maximum data in the frame
            frame_min_data.append(struct.unpack("<h", asd_file.read(2))[0])  # Minimum data in the frame
            x_offsets.append(struct.unpack("<h", asd_file.read(2))[0])  # X offset (nm)
            data_types.append(struct.unpack("<h", asd_file.read(2))[0])  # Data type
            x_tilts.append(struct.unpack("<f", asd_file.read(4))[0])  # X tilt
            y_tilts.append(struct.unpack("<f", asd_file.read(4))[0])  # Y tilt
            flag_laser_ir = struct.unpack("<?", asd_file.read(1))[0]

            # Skip 11 bytes
            asd_file.seek(asd_file.tell() + 11)

            # Read the image data
            sub = np.fromfile(asd_file, dtype=np.int16, count=header["xPixel"] * header["yPixel"])
            # Reshape the data to the correct dimensions
            preIm[k, :, :] = np.reshape(sub, (header["yPixel"], header["xPixel"])).T

        frame_numbers = np.array(frame_numbers)
        frame_max_data = np.array(frame_max_data)
        frame_min_data = np.array(frame_min_data)
        x_offsets = np.array(x_offsets)
        data_types = np.array(data_types)
        x_tilts = np.array(x_tilts)
        y_tilts = np.array(y_tilts)
        flag_laser_ir = np.array(flag_laser_ir)

        multiplier = -1 / 205.0 * header["zExtCoef"]
        im = preIm * multiplier
        print(f"multiplier: {multiplier}")
        print(f"header zExtCoef: {header['zExtCoef']}")

        # im = - preIm / 205.0 * zExtCoef

        # In my code, the equivalent of this is:
        # level * ad_range / resolution * zExtCoef
        # Level is the image pixel value, so:
        # ad_range / resolution = 1/205.0
        # resolution is usually 4096
        # so resolution / 205.0 = ad_range
        #

        rotated_frames = []
        # Flip im along the first axis (equivalent to MATLAB's flip(im))
        for frame in im:
            # Rotate the frame counterclockwise by 90 degrees
            rotated_frame = np.rot90(frame, k=-1)
            rotated_frames.append(rotated_frame)
            rotated_im = np.array(rotated_frames)

    px2nm = 1 / (header["xScanRange"] / header["xPixel"])
    return rotated_im, px2nm


# Example:
file = Path("/Users/sylvi/Downloads/121220230018.asd")
im, _ = open_asd(file)

first_frame = im[0]

sns.kdeplot(first_frame.flatten())
plt.xlabel("Height (?m)")
plt.title("Raw asd height distribution")
plt.show()

first_frame = first_frame - np.min(first_frame)

sns.kdeplot(first_frame.flatten())
plt.xlabel("Height (?m)")
plt.title("Raw asd height distribution")
plt.show()

print(f"num frames: {len(im)}")

plt.imshow(first_frame, cmap="viridis")
plt.show()

im_to_check_against = im[0]

In [ ]:
from topofileformats.asd import load_asd

In [ ]:
frames, p_to_nm, metadata = load_asd(file, channel="TP")

test_image = frames[0]

sns.kdeplot(test_image.flatten(), label="topofileformats", linewidth=3)
sns.kdeplot(im_to_check_against.flatten(), label="matlab", linewidth=3, linestyle="--")
plt.xlabel("Height (?m)")
plt.legend()
plt.show()

im_to_check_against = im_to_check_against - np.min(im_to_check_against)
test_image = test_image - np.min(test_image)
sns.kdeplot(test_image.flatten(), label="topofileformats", linewidth=3)
sns.kdeplot(im_to_check_against.flatten(), label="matlab", linewidth=3, linestyle="--")
plt.xlabel("Height (?m)")
plt.legend()
plt.show()

plt.imshow(test_image, cmap=cmap, vmin=-1, vmax=6)
plt.show()

In [ ]:
# Manually open file and read

with open(file, mode="rb") as f:
    f.seek(131)

    print(f.read(10))

In [ ]:
# turn 0x3 to hex
print(hex(0x3))

print(hex(0x00000001))

print(hex(0x00010000))

In [ ]:
# Try the other files

file = Path("../tests/resources/sample_1.asd")

im, p_to_nm, metadata = load_asd(file, channel="TP")

first_frame = im[0]

config = 

# flatten the image
filters = Filters(
    image=first_frame,
    filename="",
    pixel_to_nm_scaling=p_to_nm,

)

filters.filter_image()


plt.imshow(first_frame, cmap="viridis")
plt.show()

# plot the height distribution
sns.kdeplot(first_frame.flatten())
plt.xlabel("Height (nm)")
plt.title("Raw asd height distribution")
plt.show()